In [19]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains import VectorDBQA

In [20]:
## Read the PDFs from the folder
loader=PyPDFDirectoryLoader("./us_census")
documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents=text_splitter.split_documents(documents)
len(final_documents)

316

In [21]:
## Embedding Using Huggingface
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

In [22]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

(384,)


In [23]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [24]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [25]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E2D440B350> search_kwargs={'k': 3}


In [26]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ['MISTRAL_API_KEY']=os.getenv("MISTRAL_API_KEY")

In [27]:
# from langchain_mistralai import ChatMistralAI

# llm = ChatMistralAI(
#     model="mistral-large-latest",
#     temperature=0,
#     max_retries=2,
# )

# query = "What is the health insurance coverage?"
# response = llm.invoke(query)
# print(response)


In [28]:
from langchain import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="EleutherAI/gpt-neo-125M",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    model_kwargs={"temperature":0.1,"max_length":500}
)
query="What is the health insurance coverage?"
hf.invoke(query)

c:\Satyam_rai\Projects\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


'What is the health insurance coverage?\n\nThe health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The hea

In [29]:
from langchain_huggingface.llms import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="EleutherAI/gpt-neo-125M",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 512,
        "temperature": 0.1,
    },
)

llm = hf
response = llm.invoke(query)
print(response)

c:\Satyam_rai\Projects\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rais\.cache\huggingface\hub\models--EleutherAI--gpt-neo-125M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu
c:\Satyam_rai\Projects\venv\Lib\site-packages\transformers\generation\configuration_utils.py:628

What is the health insurance coverage?

The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health insurance coverage is a form of insurance that covers the health of the individual. The health

In [30]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [31]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [32]:
# retrievalQA = RetrievalQA.from_chain_type(
#     llm=hf,
#     chain_type="stuff",
#     retriever=retriever,
#     return_source_documents=True,
#     chain_type_kwargs={"prompt":prompt}
# )

In [33]:
qa_chain = VectorDBQA.from_chain_type(
    llm=hf,
    chain_type="stuff",                # same as your RetrievalQA
    vectorstore=vectorstore,           # pass in your FAISS vectorstore
    return_source_documents=True,      # same flag
    chain_type_kwargs={"prompt": prompt}
)

c:\Satyam_rai\Projects\venv\Lib\site-packages\langchain\chains\retrieval_qa\base.py:316: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [34]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [35]:
# Call the QA chain with our query.
docs = retriever.invoke(query)

if not docs:
    print("No relevant documents found. Try a different query.")
else:
    result = qa_chain.invoke({"query": query})
    print(result["result"])

c:\Satyam_rai\Projects\venv\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

percent (Appendix Table B-5). 
Medicaid coverage accounted 
for a portion of that difference. 
Medicaid coverage was 22.7 per-
cent in the group of states that 
expanded Medicaid eligibility and 
18.0 percent in the group of nonex-
pansion states.
CHANGES IN THE UNINSURED 
RATE BY STATE FROM 2021 
TO 2022
From 2021 to 2022, uninsured rates 
decreased across 27 states, while 
only Maine had an increase. The 
uninsured rate in Maine increased 
from 5.7 percent to 6.6 percent, 
although it remained below the 
national average. Maine’s uninsured 
rate was still below 8.0 percent, 
21 Douglas Conway and Breauna Branch, 
“Health Insurance Coverage Status and Type 
by Geography: 2019 and 2021,” 2022, <www.
census.gov/content/dam/Census/library/
publications/2022/acs/acsbr-013.pdf>.

10 U.S. Census Bureau
SUMMARY
The uninsured rate fell in 27 states 
(mainly states that